In [ ]:
import json

import boto3
import pandas as pd
# import scipy
# import numpy as np

# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_csv("sample_data/train.ende.df.short.tsv", sep='\t', index_col=0)
df["mean"] = pd.to_numeric(df["mean"], errors='coerce')

# df = df[:1000]

cands = (df.translation).tolist()
refs = (df.original).tolist()

In [ ]:
def invoke_claude_3_with_text(prompt):
    """
    Invokes Anthropic Claude 3 Sonnet to run an inference using the input
    provided in the request body.

    :param prompt: The prompt that you want Claude 3 to complete.
    :return: Inference response from the model.
    """

    # Initialize the Amazon Bedrock runtime client
    client = boto3.client(
        service_name="bedrock-runtime", region_name="us-east-1"
    )

    # Invoke Claude 3 with the text prompt
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    response = client.invoke_model(
        modelId=model_id,
        body=json.dumps(
            {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 4096,
                "temperature": 0.1,
                "messages": [
                    {
                        "role": "user",
                        "content": [{"type": "text", "text": prompt}],
                    }
                ],
            }
        ),
    )

    # Process and print the response
    result = json.loads(response.get("body").read())
    input_tokens = result["usage"]["input_tokens"]
    output_tokens = result["usage"]["output_tokens"]
    output_list = result.get("content", [])

    print("Invocation details:")
    print(f"- The input length is {input_tokens} tokens.")
    print(f"- The output length is {output_tokens} tokens.")

    print(f"- The model returned {len(output_list)} response(s):")
    for output in output_list:
        print(output["text"])

    return result

In [ ]:
prompt = """
You are an expert translator in English to German translation. You are assessing the quality of translations based on a set of criteria. 
You will rate the translation quality for each sentence pair wit a rating of RED, AMBER or GREEN. 
A RED rating is for translation which are either wholly incorrect, contain a few correct keywords but overall meaning is different from source or contains other major mistakes.
An AMBER rating is for translation which is understandable and conveys the overall meaning of the source but contains typos or grammatical errors or a translation that closely preserves the semantics of the source sentence.
A GREEN rating is for a perfect translation.

A sentence pair will be provided in the format of:
<english> source english sentence </english>
<german> translated german sentence </german>

What follows are a set of 30 examples of sentence translation pairs and their ratings (in <rating> tags) for each of the translations:
<examples>

<english>José Ortega y Gasset visited Husserl at Freiburg in 1934.</english>
<german>1934 besuchte José Ortega y Gasset Husserl in Freiburg.</german>
<rating>GREEN</rating>
<english>Meanwhile, US garrisons in Japan continually dispatched soldiers and matériel to reinforce defenders in the Pusan Perimeter.</english>
<german>Unterdessen entsandten US-Garnisonen in Japan ständig Soldaten und Matériel, um Verteidiger im Pusan Perimeter zu stärken.</german>
<rating>GREEN</rating>
<english>The HSBC also predicted that Vietnam's total GDP would surpass those of Norway, Singapore and Portugal by 2050.</english>
<german>Die HSBC prognostizierte auch, dass Vietnams gesamtes BIP das von Norwegen, Singapur und Portugal bis 2050 übertreffen würde.</german>
<rating>GREEN</rating>
<english>He criticised speculative philosophy, equating metaphysics with ideology.</english>
<german>Er kritisierte spekulative Philosophie, indem er Metaphysik mit Ideologie gleichsetzte.</german>
<rating>GREEN</rating>
<english>Jackson appointed six justices to the Supreme Court.</english>
<german>Jackson ernannte sechs Richter am Obersten Gerichtshof.</german>
<rating>GREEN</rating>
<english>On 25 February 1991, Iraqi forces fired a Scud missile at an American barracks in Dhahran, Saudi Arabia.</english>
<german>Am 25. Februar 1991 feuerten irakische Truppen eine Scud-Rakete auf eine amerikanische Kaserne in Dhahran, Saudi-Arabien.</german>
<rating>GREEN</rating>
<english>The 1990s were plagued by armed conflicts in the North Caucasus, both local ethnic skirmishes and separatist Islamist insurrections.</english>
<german>Die 1990er Jahre wurden von bewaffneten Konflikten im Nordkaukasus heimgesucht, sowohl lokalen ethnischen Auseinandersetzungen als auch separatistischen islamistischen Aufständen.</german>
<rating>GREEN</rating>
<english>3, April 1947 "Zorro's Strange Duel", West Magazine Vol.</english>
<german>3, April 1947 "Zorro 's Strange Duel", West Magazine Vol.</german>
<rating>GREEN</rating>
<english>Sara then flees; Alex later calls Sebastian, further attempting to incriminate Sara.</english>
<german>Sara flieht, Alex ruft Sebastian und versucht Sara weiter zu belasten.</german>
<rating>GREEN</rating>
<english>The Ottomans invaded Wallachia to restore Basarab Laiotă.</english>
<german>Die Osmanen drangen in die Walachei ein, um Basarab Laiotă wiederherzustellen.</german>
<rating>GREEN</rating>
<english>However, a disappointing ninth in China meant that he dropped back to sixth in the standings.</english>
<german>Eine enttäuschende Neunte in China bedeutete jedoch, dass er in der Gesamtwertung auf den sechsten Platz zurückfiel.</german>
<rating>AMBER</rating>
<english>Heavy arquebuses mounted on wagons were called arquebus à croc.</english>
<german>Schwere Arquebuses auf Waggons montiert wurden Arquebus à croc genannt.</german>
<rating>AMBER</rating>
<english>Once North Pacific salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.</english>
<german>Sobald der nordpazifische Lachs nach dem Laichen abstirbt, fressen in der Regel lokale Glatzadler fast ausschließlich Lachskörper.</german>
<rating>AMBER</rating>
<english>They engaged in crossfire at Guamaní River Bridge, Coamo and Silva Heights and finally at the Battle of Asomante.</english>
<german>Sie verübten Kreuzfeuer an der Guamaní River Bridge, Coamo und Silva Heights und schließlich an der Schlacht von Asomante.</german>
<rating>AMBER</rating>
<english>Several WHA teams courted Gretzky, notably the Indianapolis Racers and the Birmingham Bulls.</english>
<german>Mehrere WHA-Teams umworben Gretzky, insbesondere die Indianapolis Racers und die Birmingham Bulls.</german>
<rating>AMBER</rating>
<english>16 Penn State in which redshirt freshman Jim Kelly threw for 280 yards and three touchdowns in his first career start as Miami's quarterback.</english>
<german>16 Penn State, in dem Rothemd-Neuling Jim Kelly warf 280 Meter und drei Touchdowns in seiner ersten Karriere starten als Miami Quartier zurück.</german>
<rating>AMBER</rating>
<english>On 12 April 1983, the Landmarks Preservation Commission designated the 69th Regiment Armory an official New York City landmark.</english>
<german>Am 12. April 1983 ernannte die Landmarks Preservation Commission das 69. Regiment Armory zu einem offiziellen Wahrzeichen von New York City.</german>
<rating>AMBER</rating>
<english>After that, Chinese junks came to northern Borneo with cargoes of spices, bird nests, shark fins, camphor, rattan and pearls.</english>
<german>Danach kamen chinesische Junks mit Gewürzen, Vogelnestern, Haiflossen, Kampfer, Rattan und Perlen in den Norden von Borneo.</german>
<rating>AMBER</rating>
<english>Sienna Miller portrays The Baroness as one of the primary antagonists of Stephen Sommers' 2009 film G.I.</english>
<german>Sienna Miller porträtiert The Baroness als einen der Hauptantagonisten des Films G.I. von Stephen Sommers 2009</german>
<rating>AMBER</rating>
<english>The Roundhead besiegers made an effort to raise a new fort on Wall's furlong, but after some fighting with the Cavaliers they failed.</english>
<german>Die Roundhead Belagerer bemühten sich, eine neue Festung auf Wall 's Furlong zu erheben, aber nach einigen Kämpfen mit den Cavaliers scheiterten sie.</german>
<rating>AMBER</rating>
<english>In his diary, Chase wrote that the release of Mason and Slidell "…was like gall and wormwood to me.</english>
<german>In seinem Tagebuch, Chase schrieb, dass die Veröffentlichung von Mason und Slidell "war wie Galle und Wermut für mich.</german>
<rating>RED</rating>
<english>Renaissance Humanism and the Future of the Humanities.</english>
<german>Renaissance Humanism and the Future of the Humanities.</german>
<rating>RED</rating>
<english>This marked the first international bandy in Harbin.</english>
<german>Dies war der erste internationale Verband in Harbin.</german>
<rating>RED</rating>
<english>For coronation he summoned "Dadiani, Gurieli, Sharvashidze and Gelovani", but headed with Vameq II Dadiani.</english>
<german>Zur Krönung rief er "Dadiani, Gurieli, Sharvashidze und Gelovani", leitete aber Vameq II Dadiani.</german>
<rating>RED</rating>
<english>Real Time Relativity The Australian National University.</english>
<german>Real Time Relativity Die Australian National University.</german>
<rating>RED</rating>
<english>The typhoon then curved north as it rounded a subtropical ridge, although unlike most recurving tropical cyclones, a TUTT cell prevented.</english>
<german>Der Taifun kurvte dann nach Norden, als er einen subtropischen Grat umrundete, obwohl im Gegensatz zu den meisten tropischen Zyklonen eine TUTT-Zelle verhinderte.</german>
<rating>RED</rating>
<english>Their krises are gilt, and also the point of their lances.</english>
<german>Ihre Aufgänge sind vergoldet, und auch der Punkt ihrer Lanzen.</german>
<rating>RED</rating>
<english>Eight United Spanish War Veterans served as honorary pall bearers for the dead.</english>
<german>Acht United Spanish War Veterans dienten als Ehrenbürger für die Toten.</german>
<rating>RED</rating>
<english>When Carla tries to warn Maria, Tony knocks her unconscious and pulls Maria into the hostage situation.</english>
<german>Als Carla Maria zu warnen versucht, klopft Tony ihr bewusstlos an und zieht Maria in die Geiselsituation.</german>
<rating>RED</rating>
<english>Canadian expatriate musicians in the United States</english>
<german>Kanadische Wahlausländer in den Vereinigten Staaten</german>
<rating>RED</rating>

</examples>

Rate the translation quality of each of the following 10 translations:
<english>José Ortega y Gasset visited Husserl at Freiburg in 1934.</english>
<german>1934 besuchte José Ortega y Gasset Husserl in Freiburg.</german>

<english>However, a disappointing ninth in China meant that he dropped back to sixth in the standings.</english>
<german>Eine enttäuschende Neunte in China bedeutete jedoch, dass er in der Gesamtwertung auf den sechsten Platz zurückfiel.</german>

<english>In his diary, Chase wrote that the release of Mason and Slidell "…was like gall and wormwood to me.</english>
<german>In seinem Tagebuch, Chase schrieb, dass die Veröffentlichung von Mason und Slidell "war wie Galle und Wermut für mich.</german>

<english>Heavy arquebuses mounted on wagons were called arquebus à croc.</english>
<german>Schwere Arquebuses auf Waggons montiert wurden Arquebus à croc genannt.</german>

<english>Once North Pacific salmon die off after spawning, usually local bald eagles eat salmon carcasses almost exclusively.</english>
<german>Sobald der nordpazifische Lachs nach dem Laichen abstirbt, fressen in der Regel lokale Glatzadler fast ausschließlich Lachskörper.</german>

<english>They engaged in crossfire at Guamaní River Bridge, Coamo and Silva Heights and finally at the Battle of Asomante.</english>
<german>Sie verübten Kreuzfeuer an der Guamaní River Bridge, Coamo und Silva Heights und schließlich an der Schlacht von Asomante.</german>

<english>Several WHA teams courted Gretzky, notably the Indianapolis Racers and the Birmingham Bulls.</english>
<german>Mehrere WHA-Teams umworben Gretzky, insbesondere die Indianapolis Racers und die Birmingham Bulls.</german>

<english>16 Penn State in which redshirt freshman Jim Kelly threw for 280 yards and three touchdowns in his first career start as Miami's quarterback.</english>
<german>16 Penn State, in dem Rothemd-Neuling Jim Kelly warf 280 Meter und drei Touchdowns in seiner ersten Karriere starten als Miami Quartier zurück.</german>

<english>On 12 April 1983, the Landmarks Preservation Commission designated the 69th Regiment Armory an official New York City landmark.</english>
<german>Am 12. April 1983 ernannte die Landmarks Preservation Commission das 69. Regiment Armory zu einem offiziellen Wahrzeichen von New York City.</german>

<english>After that, Chinese junks came to northern Borneo with cargoes of spices, bird nests, shark fins, camphor, rattan and pearls.</english>
<german>Danach kamen chinesische Junks mit Gewürzen, Vogelnestern, Haiflossen, Kampfer, Rattan und Perlen in den Norden von Borneo.</german>


Think step by step, identifying any issues with the translation and how that affects the rating.
Output your rating assessment last.

"""

In [ ]:
invoke_claude_3_with_text(prompt)

In [ ]:
import json
import boto3
newline, bold, unbold = "\n", "\033[1m", "\033[0m"
endpoint_name = "jumpstart-dft-hf-text2text-flan-t5-20240403-134617"


def query_endpoint(payload):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, InferenceComponentName='huggingface-text2text-flan-t5-xl-20240403-134620', ContentType="application/json", Body=json.dumps(payload).encode("utf-8")
    )
    model_predictions = json.loads(response["Body"].read())
    generated_text = model_predictions[0]["generated_text"]
    print(f"Input Text: {payload['inputs']}{newline}" f"Generated Text: {bold}{generated_text}{unbold}{newline}")
# Translation
payload = {
    "inputs": "Translate to German:  My name is Arthur"
}
# query_endpoint(payload)
# # Question answering
# payload = {
#     "inputs": "Could you remind me when was the C programming language invented?",
#     "parameters": {
#         "max_new_tokens": 50,
#     },
# }
# query_endpoint(payload)
# # Recipe generation
# payload = {
#     "inputs": "Tell me the steps to make a pizza",
#     "parameters": {
#         "max_new_tokens": 200,
#         "top_k":50,
#         "top_p":0.95,
#         "do_sample":True
#     }
# }
query_endpoint(payload)
#Input must be a json
# payload = {"text_inputs":"Tell me the steps to make a pizza", "max_length":50, "num_return_sequences":3, "top_k":50, "top_p":0.95, "do_sample":True}
# 
# def query_endpoint_with_json_payload(encoded_json):
#     client = boto3.client('runtime.sagemaker')
#     response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
#     return response
# 
# query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
# 
# def parse_response_multiple_texts(query_response):
#     model_predictions = json.loads(query_response['Body'].read())
#     generated_text = model_predictions['generated_texts']
#     return generated_text
# 
# generated_texts = parse_response_multiple_texts(query_response)
# print(generated_texts)